Set up our project environment (more on this later)

In [ ]:
using Pkg
Pkg.activate(".")
Pkg.instantiate() # Only need the first time

# Intro to Julia

Topics
* Speed
* Syntax
* Autodiff
* Symbolics
* Data
* Plotting
* Modelling with Turing
* Differential Equations
* GPU

I'm going to be talking up Julia a lot during this workshop, but I want to note:
* These are my own opinions
* there's no universal "best" solution for everyone
* I am aware that it's *possible* to do most things in other languages

Instead, I hope to show you some of the areas that Julia really shines.

* I'm going to be contrasting Julia with Python a lot today since it's the language this audience is the most familiar with
* But I love Python! I used it for years, it's great at what it does, and has massive library of thoughfully developed packages

## Where we were

* Our goal should be to "do" science as efficiently and correctly as we can
* We want easy to reproduce results: free, open source, good package management 
    * pip, conda, mamba, poetry, pyenv, ...?
    * How many times have you got stuck for hours, or *days* trying to install a pacakge?
    * "It works for my collaborator, why not me?"
    * "I just deleted Python and installed Anaconda again for the fourth time"
* Code should be easy to read, write, and *understand* (no black boxes!)

People want to develop their ideas in a convenient language like MATLAB or Python, but when their problems grows they need to stop and re-write it from scratch in a "hard" but fast language like C or Fortran.
This is called the **Two Language Problem**.

This is a real impediment to research!

Since Python is so slow, most numerical libraries are actually written in C or Fortran
This leads to issues even if you never write your own pacakge:
   * The pacakge you need might not compile on your system
   * If you're using a library function, say numpy.median and you want to make your own version that's a little different, you can't (without learning all about C, compilers, combining C and Python, etc)
   * Libraries don't combine easily
   * Numbers work differently depending on if they're in a NumPy array
   * Lists vs arrays vs matrices?



Enter Julia....

## Why Julia

Julia is a "new" programming language.

* 10th birthday on Monday!
* Roughly a few 100,000 users
* Created at MIT by Alan Edelman and his graduate students, now co-founders of Julia Computing
* Free open source language but you can pay for support


* Designed from the start for science and numerical work
* Great package manager built in that just about guarantees reproducible environments
* Finally, it's *very, very fast*



Julia is an interative language just like Python. You can use it in in a terminal, in Jupyter notebooks, in Pluto Notebooks (more on this later), VS Code, etc. But it's also a compiled language like C. 
The first time you run a function, it gets compiled to fast native code ("assembly").

In [ ]:
@code_native 1 * 1

Most of this is function call overhead. There is really only one instruction executed, `imulq`
This is not the case with Python, MATLAB, R, etc.
They read the text of your program and step through line by line.

We can estimate how many instructions Python uses to multiply two integers:

In [ ]:
write("pythontest1.py", """
import time
start = time.time()
for i in range(0,1_000_000):
    i * i
end = time.time()
elapsed = (end - start)

instrs = 3.5e9 * elapsed/1_000_000

print(f"Est. instructions per Python multiply: {instrs}")
""")

# You may need to change this if you want to run these comparisons yourself
# pypath = "python"
# pypath = raw"C:\Users\William\miniconda3\python.exe"
pypath = raw"python"
run(`$pypath pythontest1.py`);

This is a good estimate of how much faster Julia is than Python.
When I run the same test using large NumPy arrays (best case) I get around 12 instructions.

In general, Julia is roughly 100-400× faster than Python, and 10-50× faster than NumPy.
For some applications like differential equations or optimization, Julia can be **350×** faster than SciPy.


<img src="https://julialang.org/assets/benchmarks/benchmarks.svg" height=400>

## So What?
* Because Julia is so fast, you can solve problems on your own laptop that would otherwise need a compute cluster
* Because it is compiled, you can write programs in natural ways without reaching for arrays all the time
* You can see how Julia packages work and make your own changes. Black boxes are bad for science!

Some words from Paul Barret:
> As an astronomer and scientific programmer at the Space Telescope Science Institute, I was one of the early developers of Numpy and matplotlib, and early advocates for its use by the astronomical community. It is now the de facto language in astronomy. However, we were aware of the two language problem at the time, but did not have the time nor the resources to implement a new language. Knowing this limitation, I was prepared to adopt Scipy's successor, if and when it arrived. Julia is that successor. Like 25 years ago, I am now advocating for Julia to become the de facto language in astonomy.

Let's dive in...

## Getting Started

### Get the notes
* Download these notes using `git` or as a zip folder from [GitHub](https://github.com/sefffal/JuliaNotes)
    * Click "Code" and then clone it, or "Download ZIP"

### Get Julia

* Download Julia: [`www.julialang.org`](https://www.julialang.org/downloads/).
Pick the current stable release.

* Install on your laptop 

Select an editor. I recommend either: VS Code or Jupyter:

#### Jupyter
1. Start Julia in a terminal `julia`
2. Type `using IJulia` and then `y` to download
4. Run `jupyterlab(dir=".")`

#### VS Code
1. Download Visual Studio Code: [`code.visualstudio.com`](https://code.visualstudio.com/download)
2. Click the "Extensions" button on the left panel and search for Julia.
3. Click "Install"


#### Terminal
The Julia REPL (terminal) is quite pleasant
1. Start Julia `julia -t auto`
2. Follow along by copying and pasting code into the terminal

# Julia Syntax

Variables

In [ ]:
a = 1
b = 3.0
c = 1//2

Mathematical expressions

In [ ]:
α = 3a + 2c

∑x = sum(3xi^2 for xi = 1:10)

Functions

In [ ]:
f(x) = 2x^2 + 3x^3 + 6

In [ ]:
f(2)

#### Mathemtical notation using $\LaTeX$
In Julia, you can type most basic LaTeX commands and hit `<Tab>`. Julia will autocomplete them and convert to Unicode.

For example, `\alpha + <Tab>` -> `α`.

There's a great font called [JuliaMono](https://juliamono.netlify.app/) that includes glyphs for almost the entire Unicode catalog. If you want your Julia code to look extra nice, give it a shot!

<img src="https://juliamono.netlify.app/assets/specimen_1.png" height=200/>

In [ ]:
∇²(σₐ) = √3 + log(σₐ)

∇²(12.0)

There are a few different ways you can print things:

In [ ]:
# Output a string (like Python print)
println("The answer to life, the universe, and everything is")

# Quick show for debugging
d = 42
@show d;

In [ ]:
# Nice logging messages
@info "The answer ... is" d
@warn "But what is the question?"

## Types
In Julia, every value has a `type`. 

In [ ]:
@show typeof(1)
@show typeof(1.0)
@show typeof("abc");

There are lots of different types in Julia. They decide what your program does!
But you almost never need to specify them and they get inferred automatically.

## Importing Libraries

The normal way to import Julia libraries is with `using SomePackage`

This loads the package (if installed).

You can then access functions from that package like `SomePackage.func()`:

In [ ]:
using Downloads
filename = Downloads.download("https://wttr.in/")

println(readuntil(filename, "┌"))

Whenever it's not ambiguous, most Julia packages `export` their key functions so you can use them without a prefix:

In [ ]:
using Statistics
mean([1,2,3])

You can load most kinds of files using the `load` function as long as you have the right package installed.
Here we'll download a an image as a PNG and load it:

In [ ]:
using Images
filename = Downloads.download("https://github.com/JuliaLang/julia-logo-graphics/blob/master/images/julia-logo-color.png?raw=true")

load(filename)

## Arrays
In Julia, vectors, matrices, etc are all just `Array`s:

In [ ]:
x = [1, 2, 3]

You can push new items into a vector using `push!`: The `!` isn't anything special. People add it to function names as a convention when that function modifies something.

In [ ]:
push!(x, 4)

Array indexing starts at 1! 

The first element is index 1, the second is 2, *wow*!

In [ ]:
q = [1,2,3,4]
@show q[1] q[2] q[3];

You can also use `begin` and `end`:

In [ ]:
@show q[begin]
@show q[end]
@show q[end - 1]

@show q[begin:end÷2]

@show q[begin:end÷2]

Just like MATLAB and Fortran, in Julia 1D arrays are column vectors by default. This is opposite from Python!

You can make matrices really easily:

In [ ]:
A = [
    1 2 3
    4 5 6
    7 8 9
]

Operators like `*`, `^2`, `sin()`, or `exp()` apply to their whole argument (unlike NumPy):

In [ ]:
A^2

In [ ]:
exp(A)

In [ ]:
A * [1, 2, 3]

## Broadcasting
If you want to apply an operation *element wise* you can prefix anything (**anything**) with a `.`:

In [ ]:
A .* [1, 2, 3]

In [ ]:
A.^2

In [ ]:
[1 2 3] .* [
             1
             2
             3 ]

In [ ]:
sin.(A)

In [ ]:
strs = [
    "A",
    "B",
    "C",
]
strs2 = ["a" "b"]
strs .* strs2

Ranges just store the start, step, and stop, but otherwise work just like any other Array:

In [ ]:
1:5

In [ ]:
y = vcat(1:5, 6:8)

In [ ]:
@show length(A)
@show size(A)
@show eachindex(A);

### Boolean Masks, Slicing

In [ ]:
a = randn(10)
mask = -0.8 .< a .< 0.8

In [ ]:
a[mask]

In [ ]:
if 2 ∈ (1,2,3,4)
    println("This is an if statement")
end

if 2 in (1,2,3,4)
    println("This way is fine too")
end

## Loops 
Loops are not bad! You can use for loops in your Julia programs as much as you want †, or you can use arrays if that makes more sense for a problem. You are free to choose!

†: If they're inside functions


In [ ]:
for i in 1:10
    println(i^2)
end

In [ ]:
for i in 1:9, j in 'A':'G'
    print(j,i," ")
    if j == 'G'
        println()
    end
end

### Multi-Threading
Using multiple threads to speed up your code is easy in Julia!
You can prefix most for loops with
```julia
@threads
```
to get an automatic speedup.
This does require you to start Julia with more than one thread (`-t auto` usually).

In [ ]:
using Base.Threads
nthreads()

In [ ]:
@threads for i in 1:100
    sum(rand(1000,1000).^2)
end

The `Distributed.jl` module also lets you use processes on other computers almost as seamlessly.
E.g.:
```julia
$> julia -p 500 # Start julia with 500 CPUs (can be spread across a cluster)
using MyCode
@everywhere myfunction() # tell each worker to run myfunction

# Or run some function across each element of an array
results = pmap(process_images, all_my_images_list)
```

## Structs
You can create your own data types using `struct`. These are really efficient and a great way to structure your program.
Watch out though, you can only define them once (have to restart Julia if changed).

In [ ]:
struct MyPoint
    x::Float64
    y::Float64
    z::Float64
end

p1 = MyPoint(1,2,3)

In [ ]:
p1.x

We can define methods of built in functions like `-` (minus)

In [ ]:
Base.:-(p1::MyPoint, p2::MyPoint) = MyPoint(p1.x - p2.x, p1.y - p2.y, p1.z - p2.z)

And now we can do math with our points:

In [ ]:
p1 = MyPoint(1,2,3)
p2 = MyPoint(1,3,4)
p2 - p1

Or even define new operators:

In [ ]:
⨳(p1::MyPoint, p2::MyPoint) = p1.x / p2.y + p1.y / p2.z + p1.z / p2.x

p1 ⨳ p2

Of course this works for any function! It doesn't have to be an operator/symbol.
This mechanism of defining standard functions for your own types (known as adding new methods) is the main reason why Julia packages work so well together!

There's no `np.cos`, `math.cos`, `sympy.cos`, `jax.cos`, `...`, there's just `cos`!

## Symbolic Calculations
You can combine your calculations with symbolic variables, a bit like SymPy or Mathematica

In [ ]:
using Symbolics
@variables u v w

expr = exp(u)^w / w

In [ ]:
simplify(expr)

## Automatic Differentiation

Most Julia code can be differentiated just like any other mathematical expression using an autodiff library. Here, we'll use ForwardDiff.jl.

This is a super power: if you have a forward modelling code and you want to compare it to data, you can get not just the $\chi^2$ but also the gradient of that $\chi^2$ with respect to all your model parameters. This can make your modelling code even more efficient!


In [ ]:
using ForwardDiff

gaussmodel(x, μ, A, σ) = A * exp(-(x-μ)^2/σ)

xdat = 0:0.5:3
dat = sin.(xdat)

meansquare(d1, d2) = sqrt(mean((d1 .- d2).^2))

fit((μ, A, σ)) = meansquare(gaussmodel.(xdat, μ, A, σ), dat)

fit((1, 0.1, 2))


In [ ]:
ForwardDiff.gradient(fit, [1, 0.1, 2])

All second order partial derviatives (the Hessian matrix):

In [ ]:
ForwardDiff.hessian(fit, [1, 0.1, 2])

## Uncertainty

The Measurements package let's you propagate uncertainty assuming Gaussian distributed errors and linear error propagataion. You can combine it with lots of other packages like Unitful for physical units with uncertainties, or simulations to propagate uncertainties through your code.

Have calculations where these assumptions don't fit? Try [MonteCarloMeasurements.jl](https://baggepinnen.github.io/MonteCarloMeasurements.jl/latest/)


In [ ]:
using Measurements
a = 2 ± 1
b = 4 ± 2

a * b

The package [IntervalArithmetic.jl](https://juliaintervals.github.io/pages/tutorials/tutorialArithmetic/) is also worth mentioning. It let's you propagate an *interval* e.g. `1..2` through your calculations and get an output interval garuanteed to include any values between [1,2]. This is a great way to test your code for floating point round off errors!

## Data
Julia has great libraries for working with tabular data. You can easily load:
 * CSV
 * Numpy .npz
 * Excel
 * R data
 * MATLAB .mat
 * SQL
 * Arrow
 
And many more...

In [ ]:
using DataFrames

df = DataFrame(
    "A" =>  1:10,
    "B" => 11:20
)

Let's load the Hipparcos-Gaia Catalog of Accelerations by Tim Brandt (2021) as a CSV file:

In [ ]:
using CSV

hgca = CSV.read("HGCA_vEDR3.csv", DataFrame)

This catalog gives the position, RV, proper motion, and astrometric acceleration of nearby stars by cross calibrating Hipparcos and GAIA.

Let's select nearby stars with the most astrometric acceleration:

In [ ]:
nearby = filter(hgca) do row
    row.parallax_gaia > 30 # About 40pc
end


sort!(nearby, [:chisq], rev=true)

# And let's pick the top  1000
beststars = nearby[1:1000, :]

## Plotting

There are two great sets of plotting packages: `Plots.jl` and `Makie.jl`. Makie is a little slow to start creates really beautiful, interactive plots, so we'll use that today.

**✋** Pick the right Makie package for your editor by uncommenting one of the following lines of code:

In [ ]:
# README!! Select one of the following...
# CairoMakie: for nice PDF exports, figures for papers
# GLMakie   : for interactive or 3D plots in a separate window
# WGLMakie  : for quick interactive plots in Jupyter

# VSCode notebook or terminal:
# using WGLMakie

# Jupyter or VSCode terminal: (not 3D or interactive)
# using CairoMakie
# CairoMakie.activate!(type="svg")

# Terminal/desktop:
# using GLMakie (have to install it yourself first, Pkg.add("GLMakie"))

if ! @isdefined Makie
    println("Error: please select one of the above plotting packaged before continuing! 👆")
end

For presenting this notebook, I would like a larger font size

In [ ]:
fontsize_theme = Theme(fontsize =20)
set_theme!(fontsize_theme)

In [ ]:
lines(1:180, sind.(1:180), axis=(xlabel="x", ylabel="y"))

In [ ]:
x = range(-π, π, length=100)
y = range(-π, π, length=100)
z = sinc.(sqrt.(x.^2 .+ y'.^2))

surface(x, y, z, colormap=:plasma)

More complex layout:

In [ ]:
fig = Figure(
    resolution=(800,800)
)

xx = π/2*randn(1000)
yy = π/2*randn(1000)

ax1 = Makie.Axis(fig[1,1], xlabel="x", ylabel="y")
scatter!(ax1, xx, yy, )

ax2 = Makie.Axis(fig[2,1], xlabel="x", ylabel="y")
h = contourf!(ax2, x, y, z)

Colorbar(fig[1:2, 2], h, label="Colorbar")


linkxaxes!(ax1, ax2)

fig

## Plotting our DataFrame
Returning to our catalog of favourite stars (that likely have companions), let's plot their positions in the sky


In [ ]:
fig,ax,pl = scatter(
    beststars.gaia_ra,
    beststars.gaia_dec,
    markersize=beststars.parallax_gaia ./ 5,
    color=log.(beststars.chisq),
    colormap=:turbo,
    axis=(;
        backgroundcolor=:black,
        gridcolor=:white,
        xlabel="RA",
        ylabel="DEC",
    )
)
Colorbar(fig[1,2], pl, label="significance")
fig

Visualize nearby stars in 3D *(this wont work with CairoMakie in Jupyter)*

In [ ]:
ρ = 1 ./ (nearby.parallax_gaia .* 1e-3)
φ = nearby.gaia_ra
θ = nearby.gaia_dec
rv = nearby.radial_velocity
rv[ismissing.(rv)] .= 0

# Spherical to Cartesian conversion
x = @. ρ * sin(φ) * cos(θ)
y = @. ρ * sin(φ) * sin(θ)
z = @. ρ * cos(φ)
# scatter(x, y, z)
fig, ax, pl = scatter(
    x,
    y,
    z,
    markersize=1000,
    color=atan.(rv),
    colormap = :bkr,
    axis = (;
        backgroundcolor=:black
    ),
    figure = (;
        resolution = (1200,900),
        backgroundcolor=:black

    )
)
scatter!(ax, [0],[0],[0], marker='⋆', color=:yellow, markersize=10000)
fig

## Statistical Modelling

The package [Turing.jl](https://turing.ml/stable/) offers a powerful language for doing Bayesian modelling.
The syntax  $ x \sim \rm{Normal(1,2)} $ is a little uncommon in astronomy, but in Statistics it's a standard way of saying that $x$ is a random variable with the probability distribution of $1 \pm 2$.

Like most Julia packages, Turing can be mixed with other packages to create something better than the sum of it's parts. For example, you can combine Turing with the Flux.jl deep learning library to create Bayesian neural nets.

We'll start by generating some simulated data of a linear relationship:

In [ ]:
N = 40
x = 20rand(N)
m = 0.4
b = 4
y = m .* x .+ b .+ 2randn(N)

scatter(
    x, y,
    axis=(
        xlabel="x",
        ylabel="y",
    )
)

Now we'll define a linear model and sample from the posterior.
Thanks to `ForwardDiff.jl` we can use the No U-Turn sampler without having to calculate gradients by hand! So not only is Julia much faster, we can also easily use more efficient algorithms.

In [ ]:
using Turing

# Define a simple Normal model with unknown mean and variance.
@model function linear_regression_1(x, y)

    # Priors on slope, itercept, and variance
    m  ~ Normal(sqrt(10))
    σ₂ ~ TruncatedNormal(0, 100, 0, Inf)
    β  ~ Normal(0, sqrt(3))

    # Equation of a line
    μ = β .+ m .* x

    # We model our y points has being drawn from a Normal distribution about that line
    y ~ MvNormal(μ, sqrt(σ₂))
end

#  Run sampler, collect results
lin_model = linear_regression_1(x, y)
chain = sample(lin_model, NUTS(0.65), 3_000)

# Or run three chains in parallel using multiple threads:
# chain = sample(model, NUTS(0.65), MCMCThreads(), 3_000, 3)

In [ ]:
# Make a traceplot
series(chain["m"]')

Let's visualize the posterior:

In [ ]:
# We'll plot our posterior samples as lines with 200 steps between 0 and 20
xpost = range(-5, 25, length=200)

# Grab 300 posterior samples at random
ii = rand(eachindex(chain["m"]), 300)
ypost = chain["m"][ii] .* xpost' .+ chain["β"][ii]

fig, ax, pl = series(
    xpost,ypost,

    solid_color=(:black, 0.02),
    label="posterior",
    axis=(
        xlabel="x",
        ylabel="y",
    ),

)

# Overplot our data
s = scatter!(ax, x, y, label="data")

Legend(fig[1,2], [s,pl], ["data","posterior"], "Legend")

xlims!(ax, low=-2, high=22)

fig

In [ ]:
hist(vec(chain["m"]), axis=(;xlabel="m"))

## Bonuses
These bonus cells demonstrate other nice packages, but I didn't include them in the installation above. You'll need to install them yourself first using e.g. `Pkg.add("PyCall")`.

## Bonus: Python in Julia

"Okay, but I have all these super specialized Python packages I need to use in my research"

`PyCall.jl` makes it easy to use Python packages inside Julia:

In [ ]:
using PyCall

np = pyimport("numpy")

a = np.array(A)

In [ ]:
@time np.mean(a)

In [ ]:
@time mean(a)

In [ ]:
py"""

print("Hello from Python!")

"""

## Bonus: Differential Equations
Differential equations are ubiquitous in the sciences. Julia has the best differential equations solving libraries (parts of SciMl).
![](https://benchmarks.sciml.ai/dev/markdown/MultiLanguage/figures/wrapper_packages_2_1.png)
This is a plot of error vs. time. Lower and to the left is better.
Julia's DifferentialEquations library is in general the fastest way to solve differential equations, including the best C libraries.

To install the DifferentialEquations library, run 
```julia
using Pkg
Pkg.add("DifferentialEquations")
```

In [ ]:
using DifferentialEquations

# Solve a simple, scalar differential equation
f(u,p,t) = 1.01*u
u0 = 1/2
tspan = (0.0,1.0)
prob = ODEProblem(f,u0,tspan)
sol = solve(prob)

[ModellingToolkit.jl](https://mtk.sciml.ai/dev/) builds on top of DifferentialEquations and Symbolics to create simulations of large/complex acausal models. It's like a mix of Mathematica, Simulink, and Modelica.

## Bonus: GPU Compute
It's very easy to get started with GPU computing in Julia since Julia can be compiled to the GPU.
GPU numerical programming in general is only supported in Windows and Linux with discrete NVidia or AMD GPUs.

I didn't include the CUDA package in this notebook, so you would need to install it yourself by running
```julia
using Pkg
Pkg.add("CUDA")
```

**✋** This package only works on Windows and Linux computers that have NVidia GPUs.

In [ ]:
if Sys.isapple()
    error("You cannot use this package on a Mac 😔.")
end

using CUDA

# Create 10000 random floats
arr = randn(Float32, 10000)

# Transfer them to the GPU
cu_arr = CuArray(arr)

# Calculate the sum of squares now using your GPU! It's that easy!
sum(cu_arr .^2)

## Bonus: Optimization
The package [Optim.jl](https://julianlsolvers.github.io/Optim.jl/stable/#user/minimization/) has a wide variety of algorithms for minimizing a function.

```julia
using Pkg
Pkg.add("Optim")
```

In [ ]:
using Optim

# Function to minimize
f(x) = (1.0 - x[1])^2 + 100.0 * (x[2] - x[1]^2)^2

# Initial guess
x0 = [0.0, 0.0]

results = optimize(f, x0)


If you want to speed up your code, you can use autodiff to automatically calculate the partial derivatives of your function!

In [ ]:
using Optim
using ForwardDiff
results_fast = optimize(f, x0, LBFGS(); autodiff = :forward)

## Bonus: Root Solving
The pacakge [Roots.jl](https://docs.juliahub.com/Roots/o0Xsi/1.3.14/) has nice routines for finding the roots of an arbitrary function.
Just like Optim.jl, you can use autodiff to speed up root finding.

```julia
using Pkg
Pkg.add("Roots")
```

In [ ]:
using Roots

f(x) =  x^5 - x + 1/2

# You can suggest a region in which to search for roots
find_zero(f, (-1.2,  -1))

## Bonus: Mixed Integer Optimization
The pacakge JuMP is a modelling language for specifying numerical optimization problems including mixed integer and non-linear problems, as well as constraints.
It supports over 44 different solvers including free Julia packages like COSMO or paid libraries like Gurobi.


```julia
using Pkg
Pkg.add(["JuMP", "GLPK"])
```

In [ ]:
using JuMP
using GLPK
model = Model(GLPK.Optimizer)
@variable(model, x >= 0)
@variable(model, 0 <= y <= 3)
@objective(model, Min, 12x + 20y)
@constraint(model, c1, 6x + 8y >= 100)
@constraint(model, c2, 7x + 12y >= 120)
print(model)
optimize!(model)

## Bonus: Distributed Processing on Compute Canada

This page by [Compute Canada](https://docs.computecanada.ca/wiki/Julia) has resources on how to use Julia on Compute Canada clusters like Cedar.

Basically, set the following environment variable:

```bash
export JULIA_DEPOT_PATH="/project/def-bob/alice/julia:$JULIA_DEPOT_PATH"
```
to store packages in your project or scatch space instead of home folder.

and 
```bash
export JULIA_PROJECT="/project/def-bon/alice/path-to-your-code"
```
to specify the project (list of packages) your code needs.

Then run:
```bash
module load julia/1.7
```

Finally, start your Julia session across multiple nodes using a slurm file like this:
```bash
#!/bin/bash
#SBATCH --ntasks=100
#SBATCH --cpus-per-task=1
#SBATCH --mem-per-cpu=1024M
#SBATCH --time=0-00:10

export JULIA_DEPOT_PATH="/project/def-bob/alice/julia:$JULIA_DEPOT_PATH"
export JULIA_PROJECT="/project/def-bon/alice/path-to-your-code"
module load julia/1.7

srun hostname -s > hostfile
sleep 5
julia --machine-file ./hostfile myprogram.jl
```

and then `sbatch submission-script.sh`.

That will run your script `myprogram.jl` automatically using 100 cores spread across the cluster.